# UnPickle Sage Data
Unpickle data created on another system with Sage.  Should then be able to create a transaction trial balances.

In [21]:
import datetime as dt
import pandas as pd

from h3_yearend import p


In [2]:
import pickle
sage_2015 = pickle.load(open('sage2015.pickle', 'rb'))
df = sage_2015.df
print(len(df))
df.head(2)

56792


,TRAN_NUMBER,TYPE,DATE,ACCOUNT_REF,ALT_REF,INV_REF,DETAILS,TAX_CODE,AMOUNT,FOREIGN_AMOUNT,BANK_FLAG,DATE_BANK_RECONCILED
0,1,SI,2015-02-03,1100,X458,55774,SGL S/F COTTON TOPPER,T1,64.44,64.44,-,NaN
1,2,SI,2015-02-03,1100,X458,55774,KNG S/F COTTON TOPPER,T1,80.10,80.10,-,NaN


# Get transaction trial balance

In [3]:
tb = sage_2015.transactional_trial_balance(dt.date(2015, 1, 1), dt.date(2016, 1, 31))
tb.head()

ACCOUNT_REF
30         687.54
1001    -74899.48
1100    124204.64
1104         0.00
1115      1479.20
dtype: object

In [4]:
assert tb.sum() == p(0)

# Working towards conversion
Sage Trial Balance to Management Trial Balance

In [7]:
conversion = {
     10: (20, 21, 30, 31, 40, 41, ),
     1001: (1001, 1004, 1254, 1256, 7906, ),
     1100: (1100, 1101, ),
     1102: (1102, ),
     1103: (1110, 1120, ),
     1115: (1104, 1115, 1117, ),
     1200: (1200, 1205, 1240, 1250, 1260, 1262, 1263, 9998, 9999, ),
     1202: (1207, 1210, 1212, 1252, ),
     1203: (1220, ),
     1204: (1230, 1232, ),
     2100: (2100, 2220, ),
     2103: (2101, ),
     2104: (2102, ),
     2105: (2105, ),
     2106: (2320, ),
     2107: (2210, 2211, ),
     2108: (2200, 2201, 2202, 2204, ),
     2109: (2109, ),
     2110: (2108, ),
     2120: (3000, ),
     2125: (3210, ),
     2126: (3200, ),
     4000: (4000, ),
     4009: (4009, ),
     4905: (4905, ),
     5000: (5000, ),
     5001: (5001, ),
     6100: (6100, ),
     6200: (6200, ),
     6201: (6201, ),
     7000: (7000, ),
     7006: (7006, ),
     7020: (7020, ),
     7100: (7100, ),
     7102: (7102, ),
     7103: (7103, ),
     7105: (7105, ),
     7200: (7200, ),
     7202: (7202, ),
     7204: (7204, ),
     7206: (7206, ),
     7906: (),
     8100: (8100, 8102, ),
     8200: (8200, ),
     8201: (7604, 8201, ),
     8204: (7901, 8204, ),
     8300: (8300, ),
     8310: (8310, ),
     8400: (8400, ),
     8402: (8402, ),
     8405: (8301, 8405, ),
     8408: (8408, ),
     8410: (8410, ),
     8414: (8414, ),
     8420: (8420, 8421, ),
     8424: (8424, ),
     8426: (8426, ),
     8430: (8430, ),
     8433: (8433, ),
     8435: (8435, ),
     8440: (7503, 8440, ),
    }

In [34]:
def convert_trial_balance(old_tb, conversion):
    index = []
    for key, value in conversion.items():
        index.append(key)
    index.sort()
    new = pd.Series([p(0)]*len(index), index = index)
    for name, value in new.iteritems():
        result = 0
        old_tb_list = conversion[name]
        for nc in old_tb_list:  # a list of accounts in the old trial balance
            try:
                result += old_tb[nc]
            except KeyError:  # Ignore where there are no entries
                pass
        new[name]=result
    return new

new_tb = convert_trial_balance(tb, conversion)
print(new_tb.head())
print('Sum of new trial balance = {}'.format(new_tb.sum()))
assert new_tb.sum() == 0

10         687.54
1001    -67891.11
1100    124204.64
1102            0
1103     61709.78
dtype: object
Sum of new trial balance = 0.00


In [26]:
[0]*2

[0, 0]